In [ ]:
!pip install --quiet sklearn

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [4]:
df_train, df_test = train_test_split(pd.read_csv('https://raw.githubusercontent.com/afonsir/cyrela-etl/main/data/parsed-data.csv'))

y_train = df_train.pop('p_dias_atraso_category')
y_test = df_test.pop('p_dias_atraso_category')

In [5]:
df_train.head()

,empresa,p_empresa,marca,p_marca,obra,p_obra,bloco,p_bloco,unidade,p_unidade,dt_venda,p_dt_venda_day,p_dt_venda_month,p_dt_venda_year,dt_chaves,p_dt_chaves_day,p_dt_chaves_month,p_dt_chaves_year,carteira_sd_gerencial,p_carteira_sd_gerencial,saldo_devedor,p_saldo_devedor,p_data_base_day,p_data_base_month,p_data_base_year,dias_atraso,p_dias_atraso,valor_pago_atualizado,p_valor_pago_atualizado,valor_pago,p_valor_pago,vgv,p_vgv
212,2876,0.908976,CYRELA,1,7470,0.752266,1,1.0,151,0.065538,2019-12-30,1.000000,1.000000,1.0095,2023-06-01,0.033333,0.500000,1.0115,1818210,0.139862,1818210.12,0.139862,1.0,0.333333,1.0105,-32,0.027634,551871.95,0.049612,507628.87,0.046148,2325838.99,0.113456
1254,2234,0.706068,CYRELA,1,9330,0.939577,1,1.0,73,0.031684,2018-12-16,0.533333,1.000000,1.0090,2021-10-01,0.033333,0.833333,1.0105,709973,0.054613,709972.58,0.054613,1.0,0.333333,1.0105,-6,0.005181,999485.18,0.089851,895683.23,0.081426,1605655.81,0.078325
1778,2094,0.661820,CYRELA,1,7910,0.796576,1,1.0,111,0.048177,2018-11-30,1.000000,0.916667,1.0090,2021-07-01,0.033333,0.583333,1.0105,430209,0.033093,430209.08,0.033093,1.0,0.333333,1.0105,-62,0.053541,198454.72,0.017841,179698.24,0.016336,609907.32,0.029752
597,3095,0.978192,LIVING,2,8722,0.878348,1,1.0,117,0.050781,2020-10-31,1.033333,0.833333,1.0100,2023-11-01,0.033333,0.916667,1.0115,758367,0.058336,758366.93,0.058336,1.0,0.333333,1.0105,-32,0.027634,54892.36,0.004935,52463.87,0.004769,810830.80,0.039553
2028,661,0.208913,CYRELA,1,2339,0.235549,1,1.0,1106,0.480035,2019-11-20,0.666667,0.916667,1.0095,2022-12-01,0.033333,1.000000,1.0110,204448,0.015727,204448.35,0.015727,1.0,0.333333,1.0105,-31,0.026770,160579.47,0.014436,148845.71,0.013531,353294.06,0.017234


In [6]:
df_train.describe()

,empresa,p_empresa,p_marca,obra,p_obra,bloco,p_bloco,unidade,p_unidade,p_dt_venda_day,p_dt_venda_month,p_dt_venda_year,p_dt_chaves_day,p_dt_chaves_month,p_dt_chaves_year,carteira_sd_gerencial,p_carteira_sd_gerencial,saldo_devedor,p_saldo_devedor,p_data_base_day,p_data_base_month,p_data_base_year,dias_atraso,p_dias_atraso,valor_pago_atualizado,p_valor_pago_atualizado,valor_pago,p_valor_pago,vgv,p_vgv
count,2248.000000,2248.000000,2248.000000,2248.000000,2248.000000,2248.0,2248.0,2248.000000,2248.000000,2248.000000,2248.000000,2248.000000,2.248000e+03,2248.000000,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000,2248.0,2.248000e+03,2.248000e+03,2248.000000,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000
mean,1781.910142,0.563183,1.688612,7170.961299,0.722151,1.0,1.0,430.493772,0.186846,0.783126,0.603722,1.009751,3.333333e-02,0.574251,1.011122,6.287159e+05,0.048363,6.338343e+05,0.048756,1.0,3.333333e-01,1.010500e+00,-78.646797,0.067916,3.172470e+05,0.028520,2.876075e+05,0.026146,9.214418e+05,0.044948
std,846.757281,0.267622,0.752988,2424.328519,0.244142,0.0,0.0,489.431455,0.212427,0.255116,0.296082,0.000512,7.495673e-16,0.271417,0.000418,7.772838e+05,0.059791,7.797351e+05,0.059980,0.0,6.829391e-15,2.176521e-14,156.170857,0.134863,5.389832e+05,0.048453,5.020102e+05,0.045637,1.083984e+06,0.052877
min,164.000000,0.051833,1.000000,2339.000000,0.235549,1.0,1.0,1.000000,0.000434,0.100000,0.083333,1.004500,3.333333e-02,0.083333,1.006000,2.160000e+02,0.000017,2.160800e+02,0.000017,1.0,3.333333e-01,1.010500e+00,-1158.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,5.000000e+04,0.002439
25%,1166.000000,0.368521,1.000000,7470.000000,0.752266,1.0,1.0,102.000000,0.044271,0.600000,0.333333,1.009500,3.333333e-02,0.416667,1.011000,2.203540e+05,0.016950,2.211748e+05,0.017013,1.0,3.333333e-01,1.010500e+00,-51.000000,0.017271,6.072245e+04,0.005459,5.685464e+04,0.005169,3.688022e+05,0.017990
50%,2043.000000,0.645702,2.000000,7869.000000,0.792447,1.0,1.0,193.000000,0.083767,0.866667,0.666667,1.010000,3.333333e-02,0.583333,1.011000,4.019825e+05,0.030922,4.041984e+05,0.031092,1.0,3.333333e-01,1.010500e+00,-32.000000,0.027634,1.596004e+05,0.014348,1.473235e+05,0.013393,5.939436e+05,0.028973
75%,2154.000000,0.680784,2.000000,8722.000000,0.878348,1.0,1.0,643.250000,0.279188,1.000000,0.916667,1.010000,3.333333e-02,0.833333,1.011500,7.625545e+05,0.058658,7.681299e+05,0.059087,1.0,3.333333e-01,1.010500e+00,-20.000000,0.044041,3.819587e+05,0.034337,3.470456e+05,0.031550,1.171475e+06,0.057145
max,3164.000000,1.000000,3.000000,9930.000000,1.000000,1.0,1.0,2304.000000,1.000000,1.033333,1.000000,1.010500,3.333333e-02,1.000000,1.012000,1.300000e+07,1.000000,1.300000e+07,1.000000,1.0,3.333333e-01,1.010500e+00,0.000000,1.000000,1.112376e+07,1.000000,1.100000e+07,1.000000,2.050000e+07,1.000000


In [7]:
y_train.value_counts()

0    995
1    878
2    375
Name: p_dias_atraso_category, dtype: int64

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
NUMERIC_COLUMNS = ['p_empresa', 'p_marca', 'p_obra', 'p_bloco', 'p_unidade',
                   'p_carteira_sd_gerencial', 'p_saldo_devedor', 'p_dias_atraso',
                   'p_valor_pago_atualizado', 'p_valor_pago', 'p_vgv',
                   'p_dt_venda_day', 'p_dt_venda_month', 'p_dt_venda_year',
                   'p_dt_chaves_day', 'p_dt_chaves_month', 'p_dt_chaves_year',
                   'p_data_base_day', 'p_data_base_month', 'p_data_base_year']

feature_columns = []

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[20, 20],
    n_classes=3)

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(df_train, y_train, training=True),
    steps=75000)

In [12]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(df_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-18T20:24:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp90dpm6c8/model.ckpt-75000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.39051s
INFO:tensorflow:Finished evaluation at 2021-06-18-20:24:12
INFO:tensorflow:Saving dict for global step 75000: accuracy = 0.57733333, average_loss = 0.7427933, global_step = 75000, loss = 0.7429673
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 75000: /tmp/tmp90dpm6c8/model.ckpt-75000

Test set accuracy: 0.577

